This is derived and modified from keras-lstm-addition-text.ipynb

In [49]:
import tensorflow
from tensorflow import keras

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dropout, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

import numpy as np

Make sure to check the version of TesorFlow and Kera. This is very important.

In [13]:
print(tensorflow.__version__)
print(keras.__version__)

2.7.0
2.7.0


This sample uses only label data ('y' data).

In [21]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [22]:
y_train[0]

5

In [23]:
# convert class vectors to binary class matrices
num_classes = 10

y_onehot_train = keras.utils.to_categorical(y_train, num_classes)
y_onehot_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [25]:
print(y_onehot_train[0])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [30]:
sequence_size = 2
output_size = 19

x_T = []
y_T = []
x_T_onehot=[]
y_T_onehot=[]

sampling_size=3
batch_size = sampling_size

x = x_train
y = y_train
y_onehot = y_onehot_train

for _ in range(sequence_size):
    y_onehot_batch=[]
    y_batch=[]
    for i in range(batch_size):
        idx = np.random.randint(0, len(x))
        # y batch
        y_batch.append(y[idx])

        # one hot encoding
        y_onehot_batch.append(y_onehot[idx])

        print(y[idx])
        print(y_onehot[idx])

    # x list of sequence
    x_T.append(y_batch)
    x_T_onehot.append(y_onehot_batch)

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
4
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
9
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
0
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [34]:
y_T=np.asarray(x_T[0]) + np.asarray(x_T[1])
print(y_T)

[10  4 14]


In [36]:
print(x_T[0])
print(x_T[1])

[5, 4, 9]
[5, 0, 5]


In [56]:
sequence_size = 2
output_size = 19

def sampling_data(x, y, y_onehot, sampling_size=50000):
    x_T = []
    y_T = []
    x_T_onehot=[]
    y_T_onehot=[]

    batch_size = sampling_size
    for _ in range(sequence_size):
        y_onehot_batch=[]
        y_batch=[]
        for i in range(batch_size):
            idx = np.random.randint(0, len(x))
            # y batch
            y_batch.append(y[idx])

            # one hot encoding
            y_onehot_batch.append(y_onehot[idx])

            #print(y[idx])
            #print(y_onehot[idx])

        # x list of sequence
        x_T.append(y_batch)
        x_T_onehot.append(y_onehot_batch)

    y_T = np.asarray(x_T[0]) + np.asarray(x_T[1])
    #print(y_T)
    #one hot encording
    y_T_onehot = keras.utils.to_categorical(y_T, output_size)


    return x_T,y_T,x_T_onehot, y_T_onehot

In [63]:
#x_T,y_T,x_T_onehot,y_T_onehot = sampling_data(x_train, y_train, y_onehot_train, 100)

# Default sampling size is 50,000.
x_T,y_T,x_T_onehot,y_T_onehot = sampling_data(x_train, y_train, y_onehot_train)

In [64]:
x_T_onehot = np.array(x_T_onehot)

In [65]:
y_T_onehot = np.array(y_T_onehot)

In [66]:
print(x_T_onehot.shape)
x_T_onehot = np.transpose(x_T_onehot, (1, 0, 2))
print(np.shape(x_T_onehot))

(2, 50000, 10)
(50000, 2, 10)


In [67]:
print(np.shape(y_T_onehot))

(50000, 19)


In [68]:
#from tensorflow.keras.models import Sequential
#from keras.layers.core import Dense, Activation
#from keras.layers.recurrent import LSTM
#from keras.optimizers import Adam

seq_length = 2
n_in = 10
n_hidden = 128
n_out = 19

model=Sequential()
model.add(LSTM(units=n_hidden, input_shape=(seq_length, n_in)))
model.add(Dense(units=n_out))
model.add(Activation('softmax'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


import numpy as np

#query = [[[1,0,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]],[[0,1,0,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]]]
#answer = [[0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

#query = np.array(query)
#answer = np.array(answer)

batch_size=20

query = x_T_onehot
answer = y_T_onehot

model.fit(query, answer,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.1,
          )
 


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
2250/2250 [==============================] - 11s 4ms/step - loss: 0.9345 - accuracy: 0.7508 - val_loss: 0.0825 - val_accuracy: 1.0000
Epoch 2/10
2250/2250 [==============================] - 9s 4ms/step - loss: 0.0234 - accuracy: 1.0000 - val_loss: 0.0055 - val_accuracy: 1.0000
Epoch 3/10
2250/2250 [==============================] - 9s 4ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 4/10
2250/2250 [==============================] - 9s 4ms/step - loss: 6.2811e-04 - accuracy: 1.0000 - val_loss: 3.0178e-04 - val_accuracy: 1.0000
Epoch 5/10
2250/2250 [==============================] - 9s 4ms/step - loss: 1.7293e-04 - accuracy: 1.0000 - val_loss: 8.6616e-05 - val_accuracy: 1.0000
Epoch 6/10
2250/2250 [==============================] - 9s 4ms/step - loss: 5.0522e-05 - accuracy: 1.0000 - val_loss: 2.5881e-05 - val_accuracy: 1.0000
Epoch 7/10
2250/2250 [==============================] - 9s 4ms/step - loss: 1.5258e-05 - accuracy: 1.0000 - val

In [79]:
query = [[[0,0,1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0,0,0]]]
answer = model.predict(query)
print (np.argmax(answer))

query = [[[0,0,0,0,0,1,0,0,0,0],[0,0,0,0,0,0,0,0,1,0]]]
answer = model.predict(query)
print (np.argmax(answer))

3
13
